In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Step 1: Import and Preprocessing
# Importing the dataset
dataset = pd.read_csv('data/Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [3]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
X.nunique()

CreditScore         460
Geography             3
Gender                2
Age                  70
Tenure               11
Balance            6382
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    9999
dtype: int64

In [5]:
X.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'],
      dtype='object')

In [6]:
one_hot_columns = [col for col in X.columns if col not in  ['CreditScore','Age','Balance', 'EstimatedSalary']]

In [7]:
X = pd.get_dummies(X, columns = one_hot_columns)

In [8]:
# Convert Ag in buckets
X['Age'] = pd.cut(X['Age'], 10)

In [9]:
X = pd.get_dummies(X,columns = ['Age'])

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X),columns = X.columns)

In [12]:
# Feature Scaling
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [17]:
# There are two ways of initializing a neiral network:
# 1. by defining the sequence of layers
# 2. by defining the graph

# In this problem, we will intialize it by defining the seqquence of layers

# Step 2: Import libraries
import keras
from keras.models import Sequential
from keras.layers import Dense

# Step 3: Initialize
classifier = Sequential()

# Step 4: Add layers

# Input layer (designated by input_dim = 11) and first hidden layer
classifier.add(Dense(units = 19 , kernel_initializer = 'uniform', activation = 'relu', input_dim = 37))
# second hidden layer
classifier.add(Dense(units = 19 , kernel_initializer = 'uniform', activation = 'relu'))
# output layer
classifier.add(Dense(units = 1 , kernel_initializer = 'uniform', activation = 'sigmoid'))

# Step 5: compile ann - apply stochastic gradient descent
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Step 6: Fit the model:
classifier.fit(X_train,y_train, batch_size = 10, epochs = 10)

Epoch 1/10
7500/7500 [==============================] - 2s 211us/step - loss: 0.4166 - accuracy: 0.8027
Epoch 2/10
7500/7500 [==============================] - 1s 165us/step - loss: 0.3735 - accuracy: 0.8523
Epoch 3/10
7500/7500 [==============================] - 1s 164us/step - loss: 0.3647 - accuracy: 0.8593
Epoch 4/10
7500/7500 [==============================] - 1s 168us/step - loss: 0.3596 - accuracy: 0.8616
Epoch 5/10
7500/7500 [==============================] - 1s 164us/step - loss: 0.3533 - accuracy: 0.8613
Epoch 6/10
7500/7500 [==============================] - 1s 167us/step - loss: 0.3480 - accuracy: 0.8640
Epoch 7/10
7500/7500 [==============================] - 1s 166us/step - loss: 0.3426 - accuracy: 0.8644
Epoch 8/10
7500/7500 [==============================] - 1s 171us/step - loss: 0.3365 - accuracy: 0.8659
Epoch 9/10
7500/7500 [==============================] - 1s 164us/step - loss: 0.3341 - accuracy: 0.8663
Epoch 10/10
7500/7500 [==============================] - 1s 164u

In [14]:
# shape of weights
for elements in classifier.get_weights():
    print(elements.shape)

(37, 19)
(19,)
(19, 19)
(19,)
(19, 1)
(1,)


In [15]:
# Step 7: Make Predictions
y_pred = classifier.predict(X_test)
# y_pred is probabolity --> convert it into class prediction to get y_pred_2 
y_pred_2 = pd.DataFrame(y_pred).apply(lambda row: 1 if row[0]>0.5 else 0, axis = 1)

In [16]:
# Step 8: Accuracy
import sklearn.metrics as metrics
print(f'accuracy: {metrics.accuracy_score(y_test, y_pred_2)}')
print(f'f1_score: {metrics.f1_score(y_test, y_pred_2)}')
print(f'precision: {metrics.precision_score(y_test, y_pred_2)}')
print(f'recall: {metrics.recall_score(y_test, y_pred_2)}')

accuracy: 0.8424
f1_score: 0.5135802469135803
precision: 0.7027027027027027
recall: 0.4046692607003891


# Step 9: Evaluating the ANN:
Judging our models's performance on one accuracy and one test set is not the best way to evaluate the model. Changing the test set will change the accuracy of our model slightly. To curb this issue, we will use k-fold cross validation.

!['cv'](images/cv.png)

### Cross validation steps:
1. Train on k-1 folds
2. Test on remaining one
3. Take mean of all k accuracies
4. Find standard deviation of all accuracies
5. Based on accuracy and standard deviations, we can see which of the following cases our model satisfies:

!['bias_variance'](images/bias_variance.JPG)

### Implementation steps:

1. cross_val_score is sklearn function
2. create a keras wrapper for sklearn so that the keras classifier can be used in sklearn cross_val_score
3. 

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [26]:
# defining the function to be passed to KerasClassifier to convert it to sklearn classifier
# only define the nn architecture. training and testing will be done by cross_val_Score
def nn_architecture():
    #initialize
    classifier = Sequential()
    # add input and first hidden layer
    classifier.add(Dense(units = 19, activation = 'relu', kernel_initializer= 'uniform' , input_dim =  37))
    # add second hidden layer
    classifier.add(Dense(units = 19, activation = 'relu', kernel_initializer= 'uniform' ))
    # add output layer
    classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer= 'uniform'))
    # compile
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [32]:
classifier_2 = KerasClassifier(build_fn= nn_architecture, batch_size = 10, nb_epoch = 100)

In [35]:
accuracies = cross_val_score(estimator = classifier_2,X = X_train, y =  y_train, cv = 10, n_jobs=-1)

In [39]:
avg_accuracy = accuracies.mean()
std_accuracy = accuracies.std()

print(f'accuracy = {avg_accuracy*100:.2f}% +/- {std_accuracy*100:.2f}%')

accuracy = 85.01% +/- 1.08%
